# LLM Iteration 7

**Aim**

Change the length of stay of patients in the ward depending on the patient type. The Stroke patients will continue to be modelled as one patient type, i.e. not considering the different lengths of stay if they go to early supported discharge or are a mortality case.

**Prompt**

The code below has been designed to model a stroke ward based on the following brief:

Code a discrete-event simulation model of a stroke ward in a hospital in python 3.10 and simpy 4.
Please code the full model specified. Do not return a simplified version. Show all code. 

All time units in the model are in days.
Each patient in the model has a unique identifier. The first patient to arrive has an identifier of 0. For each subsequent patient increment the identifier by 1.

Patients to arrive in a stroke ward that admits four different patient types: Stroke, TIA, Complex Neurological or Other. Each patient type had a different interarrival time distribution and each patient type arrival is independent of the other patient types. There are 10 beds on the stroke ward. The patient should only be admitted if there is an available bed. The patient holds the bed until their length of stay has elapsed. After the length of stay has elapsed, the patient leaves the model. 

All patient types must have their own generator function. 

The model should print out useful information after each event. 
The model should include a user settable run length. This defaults to 2 months. 

The interarrival time distribution and parameters of patients are dependent on the patient type. This describes the average time between required admissions. For each distribution time is measured in days. 
Stroke = Exponential: 1.2
TIA = Exponential: 9.3
Complex Neurological = Exponential: 3.6
Other = Exponential: 3.2

Numpy should be used for sampling. Each interarrival distribution should have its own numpy.random.Generator object. 

```python
import simpy
import numpy as np

# Define the parameters
NUM_BEDS = 10
LENGTH_OF_STAY = 7
RUN_LENGTH = 60  # Default run length in days (2 months)
INTERARRIVAL_TIMES = {
    'Stroke': 1.2,
    'TIA': 9.3,
    'Complex Neurological': 3.6,
    'Other': 3.2
}

class StrokeWard:
    def __init__(self, env):
        self.env = env
        self.beds = simpy.Resource(env, capacity=NUM_BEDS)
        self.patient_count = 0
        self.wait_times = []

    def admit_patient(self, patient):
        arrival_time = self.env.now
        with self.beds.request() as bed:
            yield bed
            wait_time = self.env.now - arrival_time
            self.wait_times.append(wait_time)
            print(f"Patient {patient.id} of type {patient.type} admitted at day {round(self.env.now, 2)}")
            yield self.env.timeout(LENGTH_OF_STAY)
            print(f"Patient {patient.id} of type {patient.type} discharged at day {round(self.env.now, 2)}")

class Patient:
    def __init__(self, id, type):
        self.id = id
        self.type = type

def patient_generator(env, ward, interarrival_time, patient_type, random_seed=None):
    rng = np.random.default_rng(random_seed)
    while True:
        yield env.timeout(rng.exponential(interarrival_time))
        patient = Patient(ward.patient_count, patient_type)
        ward.patient_count += 1
        env.process(ward.admit_patient(patient))

# Setup and start the simulation
env = simpy.Environment()
ward = StrokeWard(env)

# Initialize patient generators for each patient type with different interarrival times and random seeds
env.process(patient_generator(env, ward, INTERARRIVAL_TIMES['Stroke'], 'Stroke', random_seed=42))
env.process(patient_generator(env, ward, INTERARRIVAL_TIMES['TIA'], 'TIA', random_seed=10))
env.process(patient_generator(env, ward, INTERARRIVAL_TIMES['Complex Neurological'], 'Complex Neurological', random_seed=20))
env.process(patient_generator(env, ward, INTERARRIVAL_TIMES['Other'], 'Other', random_seed=30))

env.run(until=RUN_LENGTH)

# Calculate and print the mean wait time for a bed at the end of the run
mean_wait_time = sum(ward.wait_times) / len(ward.wait_times) if ward.wait_times else 0
print(f"Mean wait time for a bed: {round(mean_wait_time, 2)} days")

```

Update this model to change the length of stay depending on the patient type. The length of stay distribution and parameters of patients are dependent on the patient type. For each distribution time is measured in days. 
Stroke = Lognormal: mean 7.4, standard deviation 8.6
TIA = Lognormal: mean 1.8, standard deviation 2.3
Complex Neurological = Lognormal: mean 4.0, standard deviation 5.0
Other = Lognormal: mean 3.8, standard deviation 5.2
The inputs for the lognormal distribution are mu and sigma and should be calculated as follows, where v is the standard deviation squared and m is the mean:
phi = math.sqrt(v + m^2)
mu = math.log(m^2/phi)
sigma = math.sqrt(math.log(phi^2/m^2))
Numpy should be used for sampling. Each length of stay distribution should have its own numpy.random.Generator object. Each random number generator should use the random seed specified for each patient type. 

**Note**

Modifications have been made to the output from the LLM, indicated by:
```python
########## MODIFICATION ##########
change
##################################
```
This is either used to add in code from previous iterations if only certain sections were updated, used to correct the LLM's code if required, or used to check the model is working as expected.

**LLM Output**

In [1]:
import simpy
import numpy as np
import math

In [2]:
# Define the parameters
NUM_BEDS = 10
RUN_LENGTH = 60  # Default run length in days (2 months)
INTERARRIVAL_TIMES = {
    'Stroke': 1.2,
    'TIA': 9.3,
    'Complex NL': 3.6,
    'Other': 3.2
}
LENGTH_OF_STAY_PARAMS = {
    'Stroke': (7.4, 8.6),
    'TIA': (1.8, 2.3),
    'Complex NL': (4.0, 5.0),
    'Other': (3.8, 5.2)
}

In [3]:
class StrokeWard:
    def __init__(self, env):
        self.env = env
        self.beds = simpy.Resource(env, capacity=NUM_BEDS)
        self.patient_count = 0
        self.wait_times = []

        ########## MODIFICATION ##########
        self.patients_in_beds = 0
        self.bed_in_use_store = []
        self.ward_admit_time = []
        self.los_store = []
        self.discharge_store = []
        ##################################

    def admit_patient(self, patient):
        arrival_time = self.env.now
        with self.beds.request() as bed:
            yield bed
            ########## MODIFICATION ##########
            bed_arrival_time = self.env.now
            self.ward_admit_time.append(bed_arrival_time)
            self.patients_in_beds += 1
            self.bed_in_use_store.append(self.patients_in_beds)
            ##################################
            wait_time = self.env.now - arrival_time
            self.wait_times.append(wait_time)
            print(f'Patient {patient.id} of type {patient.type} admitted at '
                  f'day {round(self.env.now, 2)}')
            yield self.env.timeout(patient.length_of_stay)
            
            ########## MODIFICATION ##########
            self.patients_in_beds -= 1
            bed_leave_time = self.env.now
            self.los_store.append(bed_leave_time - bed_arrival_time)
            self.discharge_store.append(patient.type)
            ##################################
            print(f'Patient {patient.id} of type {patient.type} discharged at '
                  f'day {round(self.env.now, 2)}')

class Patient:
    def __init__(self, id, type, length_of_stay):
        self.id = id
        self.type = type
        self.length_of_stay = length_of_stay

def calculate_lognormal_params(mean, std_dev):
    phi = math.sqrt(std_dev**2 + mean**2)
    mu = math.log((mean**2) / phi)
    sigma = math.sqrt(math.log((phi**2) / (mean**2)))
    return mu, sigma

def patient_generator(env, ward, interarrival_time, patient_type,
                      length_of_stay_params, random_seed=None):
    rng_interarrival = np.random.default_rng(random_seed)
    rng_length_of_stay = np.random.default_rng(random_seed)
    
    while True:
        yield env.timeout(rng_interarrival.exponential(interarrival_time))
        
        mu, sigma = calculate_lognormal_params(*length_of_stay_params)
        length_of_stay_sample = rng_length_of_stay.lognormal(mu, sigma)
        
        patient = Patient(ward.patient_count, patient_type,
                          length_of_stay_sample)
        ward.patient_count += 1
        env.process(ward.admit_patient(patient))

In [4]:
# Setup and start the simulation with updated length of stay based on patient 
# type
env = simpy.Environment()
ward = StrokeWard(env)

# Initialize patient generators for each patient type with different 
# interarrival times and length of stay parameters
env.process(patient_generator(env, ward, INTERARRIVAL_TIMES['Stroke'],
                              'Stroke', LENGTH_OF_STAY_PARAMS['Stroke'],
                              random_seed=42))
env.process(patient_generator(env, ward, INTERARRIVAL_TIMES['TIA'], 'TIA',
                              LENGTH_OF_STAY_PARAMS['TIA'], random_seed=10))
env.process(patient_generator(env, ward, INTERARRIVAL_TIMES['Complex NL'],
                              'Complex NL',
                              LENGTH_OF_STAY_PARAMS['Complex NL'],
                              random_seed=20))
env.process(patient_generator(env, ward, INTERARRIVAL_TIMES['Other'], 'Other',
                              LENGTH_OF_STAY_PARAMS['Other'], random_seed=30))

env.run(until=RUN_LENGTH)

# Calculate and print the mean wait time for a bed at the end of the run
mean_wait_time = sum(ward.wait_times) / len(ward.wait_times) if \
                 ward.wait_times else 0
print(f"\nMean wait time for a bed: {round(mean_wait_time, 2)} days")

Patient 0 of type Complex NL admitted at day 0.77
Patient 1 of type Other admitted at day 2.41
Patient 0 of type Complex NL discharged at day 2.54
Patient 2 of type Complex NL admitted at day 2.69
Patient 3 of type Stroke admitted at day 2.89
Patient 4 of type Other admitted at day 3.0
Patient 5 of type Complex NL admitted at day 3.17
Patient 6 of type Other admitted at day 4.22
Patient 6 of type Other discharged at day 5.05
Patient 7 of type Complex NL admitted at day 5.34
Patient 8 of type Other admitted at day 5.35
Patient 9 of type Stroke admitted at day 5.69
Patient 10 of type Complex NL admitted at day 5.89
Patient 11 of type Complex NL admitted at day 6.23
Patient 8 of type Other discharged at day 7.15
Patient 12 of type Complex NL admitted at day 7.15
Patient 9 of type Stroke discharged at day 7.53
Patient 13 of type TIA admitted at day 7.53
Patient 11 of type Complex NL discharged at day 7.79
Patient 4 of type Other discharged at day 7.81
Patient 13 of type TIA discharged at d

**Checks**

In [5]:
class TestCase:
    def __init__(self, env, ward):
        self.env = env
        self.ward = ward
        self.env.process(patient_generator(self.env, self.ward,
                                           INTERARRIVAL_TIMES['Stroke'],
                                           'Stroke',
                                           LENGTH_OF_STAY_PARAMS['Stroke'],
                                           random_seed=42))
        self.env.process(patient_generator(self.env, self.ward,
                                           INTERARRIVAL_TIMES['TIA'], 'TIA',
                                           LENGTH_OF_STAY_PARAMS['TIA'],
                                           random_seed=10))
        self.env.process(patient_generator(self.env, self.ward,
                                           INTERARRIVAL_TIMES['Complex NL'],
                                           'Complex NL',
                                           LENGTH_OF_STAY_PARAMS['Complex NL'],
                                           random_seed=20))
        self.env.process(patient_generator(self.env, self.ward,
                                           INTERARRIVAL_TIMES['Other'],
                                           'Other',
                                           LENGTH_OF_STAY_PARAMS['Other'],
                                           random_seed=30))

    def test_run_length(self):
        '''
        Checks that specified run length is implemented correctly.
        '''
        self.env.run(until=RUN_LENGTH)

        if self.env.now > RUN_LENGTH:
            print('\nThe simulation has continued for longer than input.')

    def test_bed_hold(self):
        '''
        Checks that patients are holding beds until they leave the model.
        '''
        self.env.run(until=RUN_LENGTH)

        for bed_use in ward.bed_in_use_store:
            if bed_use > NUM_BEDS:
                print('\nThe number of beds in use exceeded limit.')

    def test_length_of_stay(self):
        '''
        Checks if length of stay are as expected.
        '''
        self.env.run(until=RUN_LENGTH)

        print('\n')
        for los, type in zip(ward.los_store, ward.discharge_store):
            print(f'{type} => LOS: {los}')

    def test_length_of_stay_change(self, shortest_stay):
        '''
        Checks how changing the length of stay for different patinet types
        influences the patient discharges. Change the length of stay parameters
        so one patient type has a much shorter length of stay (shortest_stay).
        Checks that only a few of the other patient types are discharged and
        gives their length of stay.
        '''
        self.env.run(until=RUN_LENGTH)

        for los, type in zip(ward.los_store, ward.discharge_store):
            if type != shortest_stay:
                print(f'\nPatient type {type} stayed for {los}.')

In [6]:
# Checks run length
NUM_BEDS = 10
RUN_LENGTH = 10

env = simpy.Environment()
ward = StrokeWard(env)

test = TestCase(env, ward)
test.test_run_length()

mean_wait_time = sum(ward.wait_times) / len(ward.wait_times) if \
                 ward.wait_times else 0
print(f"\nMean wait time for a bed: {round(mean_wait_time, 2)} days")

Patient 0 of type Complex NL admitted at day 0.77
Patient 1 of type Other admitted at day 2.41
Patient 0 of type Complex NL discharged at day 2.54
Patient 2 of type Complex NL admitted at day 2.69
Patient 3 of type Stroke admitted at day 2.89
Patient 4 of type Other admitted at day 3.0
Patient 5 of type Complex NL admitted at day 3.17
Patient 6 of type Other admitted at day 4.22
Patient 6 of type Other discharged at day 5.05
Patient 7 of type Complex NL admitted at day 5.34
Patient 8 of type Other admitted at day 5.35
Patient 9 of type Stroke admitted at day 5.69
Patient 10 of type Complex NL admitted at day 5.89
Patient 11 of type Complex NL admitted at day 6.23
Patient 8 of type Other discharged at day 7.15
Patient 12 of type Complex NL admitted at day 7.15
Patient 9 of type Stroke discharged at day 7.53
Patient 13 of type TIA admitted at day 7.53
Patient 11 of type Complex NL discharged at day 7.79
Patient 4 of type Other discharged at day 7.81
Patient 13 of type TIA discharged at d

In [7]:
# Checks that is only ever 1 patient if there is only 1 bed
NUM_BEDS = 1
RUN_LENGTH = 60

env = simpy.Environment()
ward = StrokeWard(env)

test = TestCase(env, ward)
test.test_bed_hold()

mean_wait_time = sum(ward.wait_times) / len(ward.wait_times) if \
                 ward.wait_times else 0
print(f"\nMean wait time for a bed: {round(mean_wait_time, 2)} days")

Patient 0 of type Complex NL admitted at day 0.77
Patient 0 of type Complex NL discharged at day 2.54
Patient 1 of type Other admitted at day 2.54
Patient 1 of type Other discharged at day 13.84
Patient 2 of type Complex NL admitted at day 13.84
Patient 2 of type Complex NL discharged at day 21.87
Patient 3 of type Stroke admitted at day 21.87
Patient 3 of type Stroke discharged at day 28.27
Patient 4 of type Other admitted at day 28.27
Patient 4 of type Other discharged at day 33.08
Patient 5 of type Complex NL admitted at day 33.08
Patient 5 of type Complex NL discharged at day 42.76
Patient 6 of type Other admitted at day 42.76
Patient 6 of type Other discharged at day 43.59
Patient 7 of type Complex NL admitted at day 43.59
Patient 7 of type Complex NL discharged at day 46.99
Patient 8 of type Other admitted at day 46.99
Patient 8 of type Other discharged at day 48.79
Patient 9 of type Stroke admitted at day 48.79
Patient 9 of type Stroke discharged at day 50.64
Patient 10 of type 

In [8]:
# Checks LOS values
NUM_BEDS = 10
RUN_LENGTH = 10

env = simpy.Environment()
ward = StrokeWard(env)

test = TestCase(env, ward)
test.test_length_of_stay()

mean_wait_time = sum(ward.wait_times) / len(ward.wait_times) if \
                 ward.wait_times else 0
print(f"\nMean wait time for a bed: {round(mean_wait_time, 2)} days")

Patient 0 of type Complex NL admitted at day 0.77
Patient 1 of type Other admitted at day 2.41
Patient 0 of type Complex NL discharged at day 2.54
Patient 2 of type Complex NL admitted at day 2.69
Patient 3 of type Stroke admitted at day 2.89
Patient 4 of type Other admitted at day 3.0
Patient 5 of type Complex NL admitted at day 3.17
Patient 6 of type Other admitted at day 4.22
Patient 6 of type Other discharged at day 5.05
Patient 7 of type Complex NL admitted at day 5.34
Patient 8 of type Other admitted at day 5.35
Patient 9 of type Stroke admitted at day 5.69
Patient 10 of type Complex NL admitted at day 5.89
Patient 11 of type Complex NL admitted at day 6.23
Patient 8 of type Other discharged at day 7.15
Patient 12 of type Complex NL admitted at day 7.15
Patient 9 of type Stroke discharged at day 7.53
Patient 13 of type TIA admitted at day 7.53
Patient 11 of type Complex NL discharged at day 7.79
Patient 4 of type Other discharged at day 7.81
Patient 13 of type TIA discharged at d

In [9]:
# Checks patient types other than the one with the shortest stay that have
# been discharged, to check this does not fall outside expected range
NUM_BEDS = 10
RUN_LENGTH = 60
LENGTH_OF_STAY_PARAMS = {
    'Stroke': (1.5, 5.0),
    'TIA': (60, 5.0),
    'Complex NL': (60, 5.0),
    'Other': (60, 5.0)
}

env = simpy.Environment()
ward = StrokeWard(env)

test = TestCase(env, ward)
test.test_length_of_stay_change('Stroke')

mean_wait_time = sum(ward.wait_times) / len(ward.wait_times) if \
                 ward.wait_times else 0
print(f"\nMean wait time for a bed: {round(mean_wait_time, 2)} days")

Patient 0 of type Complex NL admitted at day 0.77
Patient 1 of type Other admitted at day 2.41
Patient 2 of type Complex NL admitted at day 2.69
Patient 3 of type Stroke admitted at day 2.89
Patient 4 of type Other admitted at day 3.0
Patient 5 of type Complex NL admitted at day 3.17
Patient 3 of type Stroke discharged at day 3.58
Patient 6 of type Other admitted at day 4.22
Patient 7 of type Complex NL admitted at day 5.34
Patient 8 of type Other admitted at day 5.35
Patient 9 of type Stroke admitted at day 5.69
Patient 9 of type Stroke discharged at day 5.77
Patient 10 of type Complex NL admitted at day 5.89
Patient 11 of type Complex NL admitted at day 6.23
Patient 0 of type Complex NL discharged at day 58.8
Patient 12 of type Complex NL admitted at day 58.8
Patient 6 of type Other discharged at day 59.38
Patient 13 of type TIA admitted at day 59.38

Patient type Complex NL stayed for 58.03021423749276.

Patient type Other stayed for 55.15790472870431.

Mean wait time for a bed: 7.4

In [10]:
# Test random seeds output the same result
NUM_BEDS = 10
RUN_LENGTH = 10

env = simpy.Environment()
ward1 = StrokeWard(env)
env.process(patient_generator(env, ward1, INTERARRIVAL_TIMES['Stroke'],
                              'Stroke', LENGTH_OF_STAY_PARAMS['Stroke'],
                              random_seed=1))
env.process(patient_generator(env, ward1, INTERARRIVAL_TIMES['TIA'], 'TIA',
                              LENGTH_OF_STAY_PARAMS['TIA'], random_seed=2))
env.process(patient_generator(env, ward1, INTERARRIVAL_TIMES['Complex NL'],
                              'Complex NL',
                              LENGTH_OF_STAY_PARAMS['Complex NL'],
                              random_seed=3))
env.process(patient_generator(env, ward1, INTERARRIVAL_TIMES['Other'], 'Other',
                              LENGTH_OF_STAY_PARAMS['Other'], random_seed=4))
env.run(until=RUN_LENGTH)
mean_wait_time = sum(ward.wait_times) / len(ward.wait_times) if \
                 ward.wait_times else 0
print(f"\nMean wait time for a bed: {round(mean_wait_time, 2)} days")
print('\n')

env = simpy.Environment()
ward2 = StrokeWard(env)
env.process(patient_generator(env, ward2, INTERARRIVAL_TIMES['Stroke'],
                              'Stroke', LENGTH_OF_STAY_PARAMS['Stroke'],
                              random_seed=1))
env.process(patient_generator(env, ward2, INTERARRIVAL_TIMES['TIA'], 'TIA',
                              LENGTH_OF_STAY_PARAMS['TIA'], random_seed=2))
env.process(patient_generator(env, ward2, INTERARRIVAL_TIMES['Complex NL'],
                              'Complex NL',
                              LENGTH_OF_STAY_PARAMS['Complex NL'],
                              random_seed=3))
env.process(patient_generator(env, ward2, INTERARRIVAL_TIMES['Other'], 'Other',
                              LENGTH_OF_STAY_PARAMS['Other'], random_seed=4))
env.run(until=RUN_LENGTH)
mean_wait_time = sum(ward.wait_times) / len(ward.wait_times) if \
                 ward.wait_times else 0
print(f"\nMean wait time for a bed: {round(mean_wait_time, 2)} days")

for los1, los2 in zip(ward1.los_store, ward2.los_store):
    if los1 != los2:
        print('Different patient LOS for the same random seed.')

Patient 0 of type Complex NL admitted at day 0.4
Patient 1 of type TIA admitted at day 1.21
Patient 2 of type Stroke admitted at day 1.29
Patient 3 of type Stroke admitted at day 1.66
Patient 4 of type Complex NL admitted at day 1.8
Patient 2 of type Stroke discharged at day 2.03
Patient 3 of type Stroke discharged at day 3.24
Patient 5 of type TIA admitted at day 3.24
Patient 6 of type Complex NL admitted at day 6.84
Patient 7 of type TIA admitted at day 8.02
Patient 8 of type Stroke admitted at day 8.11
Patient 9 of type Stroke admitted at day 8.55
Patient 9 of type Stroke discharged at day 8.6
Patient 10 of type Stroke admitted at day 8.69
Patient 8 of type Stroke discharged at day 8.83

Mean wait time for a bed: 7.46 days


Patient 0 of type Complex NL admitted at day 0.4
Patient 1 of type TIA admitted at day 1.21
Patient 2 of type Stroke admitted at day 1.29
Patient 3 of type Stroke admitted at day 1.66
Patient 4 of type Complex NL admitted at day 1.8
Patient 2 of type Stroke disc

**Reflection**

The model is sampling the length of stay depending on the patient type using the lognormal distribution calculation provided. The checks show that only one patient is allowed if there is only one bed. Changing the length of stay for the different patient types changed the patient type that was discharged most often, e.g. lots more of patient type Stroke were being discharged when that had the lowest length of stay, whereas very few of the other patient types were discharged.